# Practica A2 AAMD


## 1. Lectura de les dades
Primer de tot, llegirem les dades dels nostres arxius TXT i els guardarem en dataframes.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns


df_real = pd.read_csv('data/A2-real.txt', sep=';')
df_synthetic = pd.read_csv('data/A2-synthetic.txt', sep=';')

datasets = {
    "real": df_real,
    "synthetic": df_synthetic
}

df_real.head()

,BL,BD,FL,BM,S,Location,Class
0,34.5,18.1,187,2900,1,L2,C1
1,38.8,20.0,190,3950,0,L3,C1
2,45.1,14.5,207,5050,1,L2,C2
3,34.6,21.1,198,4400,0,L1,C1
4,40.6,19.0,199,4000,0,L1,C1


# 2. Pre-processament de les dades
